In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import special

## Small example for binomial coefficient nCr using scipy

In [2]:
B=special.binom(0, 0) #binom(n,r)
print(B)

1.0


## Find Span

In [3]:
def FindSpan(n_inp,degree_inp,u_inp,knot_vector_inp):
    x=knot_vector_inp[degree_inp+1]
    if (u_inp < x):
        return degree_inp
    else:
        for i,pos in enumerate(knot_vector_inp):
            if math.floor(u_inp) == pos:
                return (i)

## Derivatives of Non_Zero basis Functions

In [4]:
# Derivatives are stored in ders[k][j]
# ders[k][j] is the kth derivative of function N_(i-p+j,p)
# If k>p the derivatives are zero.
def DersBasisFuns(i,u,p,m,U):
    #Inititation of dimentions for 2D matrices
    ndu=np.zeros((p+1,p+1))
    ders=np.zeros((p+1,p+1))
    a=np.zeros((2,p+1))
    left =np.zeros(p+2)
    right =np.zeros(p+2)
    
    ndu[0][0]=1.0
    for j in range(1,p+1):
        left[j] = u - U[i+1-j]
        right[j] = U[i+j] - u
        saved=0.0
        for r in range(j):
            #Lower triangle
            ndu[j][r] = right[r+1]+left[j-r]
            temp=ndu[r][j-1]/ndu[j][r]
            #Upper triangle
            ndu[r][j] = saved+(right[r+1]*temp)
            saved=left[j-r]*temp
        ndu[j][j] = saved
    for j in range (p+1): #Load the basis functions
        ders[0][j]=ndu[j][p]
    #This secion computes the derivatives
    for r in range(p+1):
        s1=0
        s2=1 #Alternative rows in array a
        a[0][0] = 1.0
        #Loop to compute kth derivative
        for k in range(1,m+1):
            d=0.0
            rk=r-k
            pk=p-k
            if(r>=k):
                a[s2][0]=a[s1][0]/ndu[pk+1][rk]
                d=a[s2][0]*ndu[rk][pk]
            if(rk>=-1):
                j1=1
            else:
                j1=-rk
            if(r-1<=pk):
                j2=k-1
            else:
                j2=p-r
            for j in range (j1,j2+1):
                a[s2][j] =(a[s1][j]-a[s1][j-1])/ndu[pk+1][rk+j]
                d += (a[s2][j]*ndu[rk+j][pk])
            if(r<=pk):
                a[s2][k]=-a[s1][k-1]/ndu[pk+1][r]
                d+=(a[s2][k]*ndu[r][pk])
            ders[k][r]=d
            #Switch rows
            j=s1
            s1=s2
            s2=j
            #Multiply through by the correct factors
    r=p
    for k in range(1,m+1):
        for j in range(p+1):
            ders[k][j] =ders[k][j]* r
        r =r* (p-k)
    return ders

## Aders

In [5]:
#An extra W is given as input to function for convinience which contain weights of corresponding control points
def SurfaceDerivsAlgAuv(n,p,U,m,q,V,P,W,u,v,d):
    #SKL=np.zeros((p+1,q+1,3))
    SKL=np.zeros((20,20,3))
    temp=np.zeros((q+1,3))
    #print(SKL)
    #print(temp[0])
    #print(P[1][1])
    du = min(d,p)
    for k in range(p+1,d+1):
        for l in range(d-k+1):
            SKL[k][l]=0.0
    dv = min(d,q)
    for l in range(q+1,d+1):
        for k in range(d-l+1):
            SKL[k][l]=0.0 
    uspan=FindSpan(n,p,u,U)
    unders=DersBasisFuns(uspan,u,p,du,U)
    vspan=FindSpan(m,q,v,V)
    vnders=DersBasisFuns(vspan,v,q,dv,V)
    for k in range(du+1):
        for s in range(q+1):
            temp[s] = 0.0
            for r in range(p+1):
                temp[s]=temp[s] + unders[k][r]*P[uspan-p+r][vspan-q+s]*W[uspan-p+r][vspan-q+s]
            dd=min(d-k,dv)
            for l in range(dd+1):
                SKL[k][l] = 0.0
                for s in range(q+1):
                    SKL[k][l] = SKL[k][l] + vnders[l][s]*temp[s]
    return SKL #Returning Aders here

## wders

In [6]:
def SurfaceDerivsAlgWuv(n,p,U,m,q,V,P,W,u,v,d):
    #SKL=np.zeros((p+1,q+1,1)) #Have to cross check if this is right way of dimensioning SKL
    SKL=np.zeros((20,20,1))
    temp=np.zeros((q+1,1))    # Have to be because W is a scalar unlike P which is vector
    #print(SKL)
    #print(temp[0])
    #print(P[1][1])
    du = min(d,p)
    for k in range(p+1,d+1):
        for l in range(d-k+1):
            SKL[k][l]=0.0
    dv = min(d,q)
    for l in range(q+1,d+1):
        for k in range(d-l+1):
            SKL[k][l]=0.0 
    uspan=FindSpan(n,p,u,U)
    #print('USPAN',uspan)
    unders=DersBasisFuns(uspan,u,p,du,U)
    vspan=FindSpan(m,q,v,V)
    #print('VSPAN',vspan)
    vnders=DersBasisFuns(vspan,v,q,dv,V)
    for k in range(du+1):
        for s in range(q+1):
            temp[s] = 0.0
            for r in range(p+1):
                temp[s]=temp[s] + unders[k][r]*W[uspan-p+r][vspan-q+s]
            dd=min(d-k,dv)
            for l in range(dd+1):
                SKL[k][l] = 0.0
                for s in range(q+1):
                    SKL[k][l] = SKL[k][l] + vnders[l][s]*temp[s]
    return SKL #Returning wders here

## Only derivatives of NURBS Functions

In [7]:
def SurfaceDerivsAlgNURBS(n,p,U,m,q,V,P,W,u,v,d):
    #SKL=np.zeros((p+1,q+1,1)) #Have to cross check if this is right way of dimensioning SKL
    SKL=np.zeros((20,20,1))
    temp=np.zeros((q+1,1))    # Have to be because W is a scalar unlike P which is vector
    #print(SKL)
    #print(temp[0])
    #print(P[1][1])
    du = min(d,p)
    for k in range(p+1,d+1):
        for l in range(d-k+1):
            SKL[k][l]=0.0
    dv = min(d,q)
    for l in range(q+1,d+1):
        for k in range(d-l+1):
            SKL[k][l]=0.0 
    uspan=FindSpan(n,p,u,U)
    #print('USPAN',uspan)
    unders=DersBasisFuns(uspan,u,p,du,U)
    vspan=FindSpan(m,q,v,V)
    #print('VSPAN',vspan)
    vnders=DersBasisFuns(vspan,v,q,dv,V)
    for k in range(du+1):
        for s in range(q+1):
            temp[s] = 0.0
            for r in range(p+1):
                temp[s]=temp[s] + unders[k][r]
            dd=min(d-k,dv)
            for l in range(dd+1):
                SKL[k][l] = 0.0
                for s in range(q+1):
                    SKL[k][l] = SKL[k][l] + vnders[l][s]*temp[s]
    return SKL #Returning wders here

## Function to extract derivatives of NURBS Surface

In [8]:
#Functions gives SKL matrix which contain kth ans lth derivatives of the NURBS Surface function S(u,v)
#For example 1st derivative of S(u,v) wrt to xi_para is stored in SKL[1][0]
#Likewise 1st derivative of S(u,v) wrt to eta_para is stored in SKL[0][1]

def RatSurfaceDerivs(Aders,wders,d):
    #SKL=np.zeros((p+1,q+1,3))
    SKL=np.zeros((20,20,3))
    for k in range(d+1):
        for l in range(d-k+1):
            v=Aders[k][l]
            for j in range(1,l+1):
               # print('B',special.binom(l,j))
                v=v-special.binom(l,j)*wders[0][j]*SKL[k][l-j]
            for i in range(1,k+1):
                v=v-special.binom(k,i)*wders[i][0]*SKL[k-i][l]
                v2 = 0.0
                for j in range(1,l+1):
                    v2=v2+special.binom(l,j)*wders[i][j]*SKL[k-i][l-j]
                v = v-special.binom(k,i)*v2
            SKL[k][l] = v/wders[0][0]
    return SKL

## Higher (4th) Order  element p=3, q=3

In [17]:
#----------Manually change below variables--------------#
p=3
q=3
U=np.array([0., 0., 0., 0., 1., 2., 2., 2., 2.])
V=np.array([0., 0., 0., 0., 1., 2., 2., 2., 2.])
P_W=np.array([[[0,0,0,1],[0,0.25,0,1],[0,0.5,0,1],[0,0.75,0,1],[0,1,0,1]],
              [[0.25,0,0,1],[0.25,0.25,0,1],[0.25,0.5,0,1],[0.25,0.75,0,1],[0.25,1,0,1]],
              [[0.5,0,0,1],[0.5,0.25,0,1],[0.5,0.5,0,1],[0.5,0.75,0,1],[0.5,1,0,1]],
              [[0.75,0,0,1],[0.75,0.25,0,1],[0.75,0.5,0,1],[0.75,0.75,0,1],[0.75,1,0,1]],
              [[1.0,0,0,1],[1.0,0.25,0,1],[1.0,0.5,0,1],[1.0,0.75,0,1],[1.0,1,0,1]]])
d=2
ndof = 2


#-----------GAUSS POINTS---------------#
GPs_Ws = np.array([[-0.8611,-0.8611,0.1210],[-0.8611,-0.3399,0.2269],[-0.8611,0.3399,0.2269],[-0.8611,0.8611,0.1210],
                   [-0.3399,-0.8611,0.2269],[-0.3399,-0.3399,0.4253],[-0.3399,0.3399,0.4253],[-0.3399,0.8611,0.2269],
                   [0.3399,-0.8611,0.2269],[0.3399,-0.3399,0.4253],[0.3399,0.3399,0.4253],[0.3399,0.8611,0.2269],
                   [0.8611,-0.8611,0.1210],[0.8611,-0.3399,0.2269],[0.8611,0.3399,0.2269],[0.8611,0.8611,0.1210]])
#----------Need not alter below variables--------------#
n=(np.size(U)-1)-p-1
m=(np.size(V)-1)-q-1
P=P_W[:,:,0:3]
W=P_W[:,:,3]
ncpxi=np.shape(P_W)[1]  #No.of control points xi direction
ncpeta=np.shape(P_W)[1] #No.of control points eta direction
nel=(ncpxi-p-1)*(ncpeta-q-1)
ncp = ncpxi*ncpeta #Total number of control points
necp = (p+2)*(q+2) #Total number of control points per element

## Loop over gauss points

In [28]:
KG = np.zeros((ndof*ncp,ndof*ncp)) #Initiating Global K matrix
RHSG = np.zeros((ndof*ncp,1)) # Initiating Global RHS matrix
for i in range(nel):
    Ke=np.zeros((ndof*necp,ndof*necp))
    B=np.zeros((3,ndof*necp)) #Dimentions depends on type of application
    elU = np.array([0,2]) #Manually defined have to generate using connectivity functions
    elV = np.array([0,2]) #Manually defined have to generate using connectivity functions
    
    #-------------------GAUSS LOOP-------------------#
    for j in range(np.shape(GPs_Ws)[0]):
        gp = GPs_Ws[j,0:2]
        wg = GPs_Ws[j,2]
        ximas = gp[0]
        etamas = gp[1]
        xi = 0.5*((elU[1]-elU[0])*ximas + (elU[1]+elU[0]))     #Paramteric co-ordinates
        eta = 0.5*((elV[1]-elV[0])*etamas + (elV[1]+elV[0]))
        dxi_dximas = 0.5*(elU[1]-elU[0])
        deta_detamas = 0.5*(elV[1]-elV[0]) 
        J2 = dxi_dximas*deta_detamas
        #--------Evaluation of NURBS basis functions derivatives---------#
        Aders = SurfaceDerivsAlgAuv(n,p,U,m,q,V,P,W,xi,eta,d)
        wders = SurfaceDerivsAlgWuv(n,p,U,m,q,V,P,W,xi,eta,d)
        dRx = RatSurfaceDerivs(Aders,wders,d)
        dRx_dxi  =  dRx[1][0][0]
        dRx_deta =  dRx[0][1][0]
        dRy_dxi  =  dRx[1][0][1]
        dRy_deta =  dRx[0][1][1]
        J1 = np.array([[dRx_dxi,dRx_deta],
                  [dRy_dxi,dRy_deta]])
        #print('J1 Matrix : ',J1)
        J1det = (dRx_dxi*dRy_deta)-(dRx_deta*dRy_dxi)
        #print('Determinat of J1 : ',J1det)
        J1inv = np.linalg.inv(J1)
        #print('J1inv : ',J1inv)
        #---------------NURBS Basis Functions Derivatives wrt x and y-------------#
        Random = np.array([dRx[1][0][0:2],dRx[0][1][0:2]])
        #print(Random)
        dR_dxdy = np.matmul(J1inv,Random)
        print(dR_dxdy)

[[ 1.00000000e+00 -1.94823451e-18]
 [ 1.94823451e-18  1.00000000e+00]]
[[ 1.00000000e+00  2.78465553e-18]
 [-4.34721609e-18  1.00000000e+00]]
[[ 1.00000000e+00 -9.52841403e-17]
 [ 1.48751163e-16  1.00000000e+00]]
[[ 1.00000000e+00 -1.06488585e-16]
 [ 1.06488585e-16  1.00000000e+00]]
[[ 1.00000000e+00  1.04329283e-16]
 [-6.68292326e-17  1.00000000e+00]]
[[ 1.00000000e+00 -8.11002077e-17]
 [ 8.11002077e-17  1.00000000e+00]]
[[ 1.00000000e+00  1.18605838e-16]
 [-1.18605838e-16  1.00000000e+00]]
[[ 1.00000000e+00 -2.43199028e-16]
 [ 1.55783725e-16  1.00000000e+00]]
[[ 1.00000000e+00  7.19976168e-17]
 [-4.61188397e-17  1.00000000e+00]]
[[ 1.00000000e+00 -2.64474414e-17]
 [ 2.64474414e-17  1.00000000e+00]]
[[ 1.00000000e+00 -4.40028188e-19]
 [ 4.40028188e-19  1.00000000e+00]]
[[ 1.00000000e+00 -5.08596587e-16]
 [ 3.25786957e-16  1.00000000e+00]]
[[ 1.0000000e+00  1.3718718e-16]
 [-1.3718718e-16  1.0000000e+00]]
[[ 1.00000000e+00 -1.62091566e-16]
 [ 2.53046402e-16  1.00000000e+00]]
[[ 1.00000

## Rough implementation of Stiffness Matrix generation

### Parameters

In [11]:
##*****Knot vectors must be integers****## cannot be 0.5   etc........
##*******RESTRICTION***************##
# U = np.array([0., 0., 0., 0.,0.5, 1., 1., 1., 1.])
# V = np.array([0., 0., 0., 0.,0.5, 1., 1., 1., 1.])
U = np.array([0., 0., 0., 0.,1., 2., 2., 2., 2.]) 
V = np.array([0., 0., 0., 0.,1., 2., 2., 2., 2.]) 
p=3
q=3
n=(np.size(U)-1)-p-1
m=(np.size(V)-1)-q-1
P_W_T=np.array([[[0,0,0,1],[0.25,0,0,1],[0.5,0,0,1],[0.75,0,0,1],[1,0,0,1]],
             [[0,0.25,0,1],[0.25,0.25,0,1],[0.5,0.25,0,1],[0.75,0.25,0,1],[1,0.25,0,1]],
             [[0,0.5,0,1],[0.25,0.5,0,1],[0.5,0.5,0,1],[0.75,0.5,0,1],[1,0.5,0,1]],
             [[0,0.75,0,1],[0.25,0.75,0,1],[0.5,0.75,0,1],[0.75,0.75,0,1],[1,0.75,0,1]],
             [[0,1,0,1],[0.25,1,0,1],[0.5,1,0,1],[0.75,1,0,1],[1,1,0,1]]])
P_W=np.array([[[0,0,0,1],[0,0.25,0,1],[0,0.5,0,1],[0,0.75,0,1],[0,1,0,1]],
              [[0.25,0,0,1],[0.25,0.25,0,1],[0.25,0.5,0,1],[0.25,0.75,0,1],[0.25,1,0,1]],
              [[0.5,0.,0,1],[0.5,0.25,0,1],[0.5,0.5,0,1],[0.5,0.75,0,1],[0.5,1,0,1]],
              [[0.75,0,0,1],[0.75,0.25,0,1],[0.75,0.5,0,1],[0.75,0.75,0,1],[0.75,1,0,1]],
              [[1.0,0,0,1],[1.0,0.25,0,1],[1.0,0.5,0,1],[1.0,0.75,0,1],[1.0,1,0,1]]])
P=P_W[:,:,0:3]
W=P_W[:,:,3]
d=3
print(P[0][1]*W[0][0])

[0.   0.25 0.  ]


### Stiffness matrix for 2D case with u1 and u2 dof

In [12]:
ncpx =5 # No.control points in x direction
ncpy =5 # No.control points in y direction
ncp = ncpx*ncpy
ndof = 2
# Gauss points and corresponding weights in a single matrix
# This matrix will depend on the type of problem. So have to change in parameters. 
# Or alternatively create a function
GPs_Ws = np.array([[-0.5774,-0.5774,1],[0.5774,-0.5774,1],[0.5774,0.5774,1],[-0.5774,0.5774,1]])
#GPs_Ws[:,2] gauss weights
#GPs_Ws[:,0:2] gauss points

#Have to see how to make it work for multiple elements
nel = (ncpx-p-1)*(ncpy-q-1) #Total number of elements
necp =(p+2)*(q+2) #Total number of control points in an element 
#Caution have to check on what it depends
gp=GPs_Ws[0,0:2]

In [13]:
KG = np.zeros((ndof*ncp,ndof*ncp)) #Initiating Global K matrix
RHSG = np.zeros((ndof*ncp,1)) # Initiating Global RHS matrix
for i in range(nel):
    Ke=np.zeros((ndof*necp,ndof*necp))
    B=np.zeros((3,ndof*necp)) #Dimentions depends on type of application
    elU = np.array([0,2]) #Manually defined have to generate using connectivity functions
    elV = np.array([0,2]) #Manually defined have to generate using connectivity functions
    
    for j in range(np.shape(GPs_Ws)[0]): #Computes number of gauss points from GP matrix provided
        gp = GPs_Ws[j,0:2]
        wg = GPs_Ws[j,2]
        xi_para = gp[0]
        eta_para = gp[1]
        xi = 0.5*((elU[1]-elU[0])*xi_para + (elU[1]+elU[0]))
        eta = 0.5*((elV[1]-elV[0])*eta_para + (elV[1]+elV[0]))
        dxi_dxi_para = 0.5*(elU[1]-elU[0])
        deta_deta_para = 0.5*(elV[1]-elV[0]) 
        J2 = dxi_dxi_para*deta_deta_para
        
        #--------Evaluation of NURBS basis functions derivatives---------#
        # We are finding the derivative at gauss points
        u=xi 
        #************What is point in sending xi and eta values what does that mena here????************#
        v=eta
        Aders = SurfaceDerivsAlgAuv(n,p,U,m,q,V,P,W,u,v,d)
        wders = SurfaceDerivsAlgWuv(n,p,U,m,q,V,P,W,u,v,d)
        R = RatSurfaceDerivs(Aders,wders,d)
        print(R[0][1][0])
        jacob = np.array([[R[1][0][0],R[0][1][0]],
                          [R[1][0][1],R[0][1][1]]])
        J1=np.linalg.det(jacob)

7.461393830399117e-18
-6.991143896556418e-17
0.0
0.0


## 2nd Order Element

In [14]:
##*****Knot vectors must be integers****## cannot be 0.5   etc........
##*******RESTRICTION***************##
U = np.array([0., 0., 1.,1.]) 
V = np.array([0., 0., 1.,1.]) 
p=1
q=1
n=(np.size(U)-1)-p-1
m=(np.size(V)-1)-q-1
#***********Page 116 for control points array definition ************#
P_W=np.array([[[0,0,0,1],[0,1,0,1]],
           [[1,0,0,1],[1,1,0,1]]])
P=P_W[:,:,0:3]
W=P_W[:,:,3]
d=1
ncpx =2 # No.control points in x direction
ncpy =2 # No.control points in y direction
ncp = ncpx*ncpy
ndof = 2
# Gauss points and corresponding weights in a single matrix
# This matrix will depend on the type of problem. So have to change in parameters. 
# Or alternatively create a function
GPs_Ws = np.array([[-0.5774,-0.5774,1],[0.5774,-0.5774,1],[0.5774,0.5774,1],[-0.5774,0.5774,1]])
#GPs_Ws[:,2] gauss weights
#GPs_Ws[:,0:2] gauss points

#Have to see how to make it work for multiple elements
nel = (ncpx-p-1)*(ncpy-q-1)+1 #Total number of elements
print('Number of elements are:', nel)
necp =(p+2)*(q+2) #Total number of control points in an element 
#Caution have to check on what it depends
gp=GPs_Ws[0,0:2]

Number of elements are: 1


In [15]:
KG = np.zeros((ndof*ncp,ndof*ncp)) #Initiating Global K matrix
RHSG = np.zeros((ndof*ncp,1)) # Initiating Global RHS matrix
for i in range(nel):
    Ke=np.zeros((ndof*necp,ndof*necp))
    B=np.zeros((3,ndof*necp)) #Dimentions depends on type of application
    elU = np.array([0,1]) #Manually defined have to generate using connectivity functions
    elV = np.array([0,1]) #Manually defined have to generate using connectivity functions
    
    for j in range(np.shape(GPs_Ws)[0]): #Computes number of gauss points from GP matrix provided
        gp = GPs_Ws[j,0:2]
        wg = GPs_Ws[j,2]
        xi_para = gp[0]
        eta_para = gp[1]
        xi = 0.5*((elU[1]-elU[0])*xi_para + (elU[1]+elU[0]))
        eta = 0.5*((elV[1]-elV[0])*eta_para + (elV[1]+elV[0]))
        dxi_dxi_para = 0.5*(elU[1]-elU[0])
        deta_deta_para = 0.5*(elV[1]-elV[0]) 
        J2 = dxi_dxi_para*deta_deta_para
        
        #--------Evaluation of NURBS basis functions derivatives---------#
        # We are finding the derivative at gauss points
        u=xi 
        #************What is point in sending xi and eta values what does that mena here????************#
        v=eta
        Aders = SurfaceDerivsAlgAuv(n,p,U,m,q,V,P,W,xi,eta,d)
        wders = SurfaceDerivsAlgWuv(n,p,U,m,q,V,P,W,xi,eta,d)
        R = RatSurfaceDerivs(Aders,wders,d)
        R_NURBS = SurfaceDerivsAlgNURBS(n,p,U,m,q,V,P,W,xi,eta,d)
        print(R[0][1])
        J1 = np.array([[R[1][0][0],R[0][1][0]],
                          [R[1][0][1],R[0][1][1]]])
        J1inv=np.linalg.inv(jacob)
        J1det=np.linalg.det(jacob)
        R_dxideta = np.array([R_NURBS[1][0],R_NURBS[0][1]])
        R_dxdy = np.matmul(J1inv,R_dxideta)
        
        

[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
[0. 1. 0.]
